In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLPScholarStructuredBenchmark

benchmark = DBLPScholarStructuredBenchmark(data_dir_path="../data/")
benchmark

17:55:33 INFO:Extracting DBLP-Scholar-Structured...
17:55:33 INFO:Reading DBLP-Scholar-Structured row_dict...
17:55:33 INFO:Reading DBLP-Scholar-Structured train.csv...
17:55:33 INFO:Reading DBLP-Scholar-Structured valid.csv...
17:55:33 INFO:Reading DBLP-Scholar-Structured test.csv...


<DBLPScholarStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-GoogleScholar/dblp_scholar_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'authors', 'venue', 'year']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'authors': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'venue': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_venue': {
        'source_attr': 'venue',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'year': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

17:55:34 INFO:For attr=title, computing actual max_str_len
17:55:34 INFO:For attr=title, using actual_max_str_len=70
17:55:35 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:55:39 INFO:For attr=authors, computing actual max_str_len
17:55:39 INFO:actual_max_str_len=33 must be even to enable NN pooling. Updating to 34
17:55:39 INFO:For attr=authors, using actual_max_str_len=34
17:55:39 INFO:For attr=venue, computing actual max_str_len
17:55:39 INFO:actual_max_str_len=45 must be even to enable NN pooling. Updating to 46
17:55:39 INFO:For attr=venue, using actual_max_str_len=46
17:55:39 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:55:43 INFO:For attr=year, computing actual max_str_len
17:55:43 INFO:For attr=year, using actual_max_str_len=6


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=70, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    eval_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name)
)

17:55:43 INFO:GPU available: True, used: True
17:55:43 INFO:TPU available: None, using: 0 TPU cores
17:55:43 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

17:55:43 INFO:Train pair count: 3207
17:55:43 INFO:Valid pair count: 1070
17:55:43 INFO:Test pair count: 1070
17:55:45 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 31.1 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
15.0 M    Trainable params
16.2 M    Non-trainable params
31.1 M    Total params


17:55:46 WARNING:true_pair_set has 3066 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.3880322277545929,
 'semantic_title': 0.309906929731369,
 'authors': 0.16600309312343597,
 'venue': 0.06950557976961136,
 'semantic_venue': 0.03297638148069382,
 'year': 0.03357579559087753}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.16475125337446972,
 'valid_f1_at_0.5': 0.6869340232858991,
 'valid_f1_at_0.7': 0.9378378378378379,
 'valid_f1_at_0.9': 0.9225490196078432,
 'valid_pair_entity_ratio_at_0.3': 6.244094488188976,
 'valid_pair_entity_ratio_at_0.5': 1.0614173228346457,
 'valid_pair_entity_ratio_at_0.7': 0.6036745406824147,
 'valid_pair_entity_ratio_at_0.9': 0.5091863517060368,
 'valid_precision_at_0.3': 0.08978562421185372,
 'valid_precision_at_0.5': 0.5252225519287834,
 'valid_precision_at_0.7': 0.9052173913043479,
 'valid_precision_at_0.9': 0.9701030927835051,
 'valid_recall_at_0.3': 0.9981308411214953,
 'valid_recall_at_0.5': 0.9925233644859813,
 'valid_recall_at_0.7': 0.9728971962616823,
 'valid_recall_at_0.9': 0.8794392523364486}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

18:03:03 INFO:Train pair count: 3207
18:03:03 INFO:Valid pair count: 1070
18:03:03 INFO:Test pair count: 1070


[{'test_f1_at_0.3': 0.16677086589559237,
  'test_f1_at_0.5': 0.6786632390745501,
  'test_f1_at_0.7': 0.9447740757644911,
  'test_f1_at_0.9': 0.9213147410358566,
  'test_pair_entity_ratio_at_0.3': 6.14248297537978,
  'test_pair_entity_ratio_at_0.5': 1.069669984284966,
  'test_pair_entity_ratio_at_0.7': 0.5872184389732844,
  'test_pair_entity_ratio_at_0.9': 0.49135673127291773,
  'test_precision_at_0.3': 0.09099437148217636,
  'test_precision_at_0.5': 0.5171400587659157,
  'test_precision_at_0.7': 0.9232827832292596,
  'test_precision_at_0.9': 0.9861407249466951,
  'test_recall_at_0.3': 0.997196261682243,
  'test_recall_at_0.5': 0.9869158878504672,
  'test_recall_at_0.7': 0.9672897196261683,
  'test_recall_at_0.9': 0.8644859813084113}]